In [29]:
from Moduli.ClassDictionary import ClassDictionary as cDict
from Moduli.Dataset import ScenesDatabase as sd
from Moduli.BellomoClassifier import BellomoClassifier  

from torchvision import transforms

In [30]:
m = [0.485, 0.456, 0.408]
s = [0.229, 0.224, 0.225] 

test_transform = transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor(),
    transforms.Normalize(m,s)
])

In [31]:
test = sd('Bellomo/Training/','Bellomo/Training/training.txt', transform=test_transform ,split= "second") 

In [33]:
import numpy as np
import torch
from torch.utils.data import DataLoader
import tkinter as tk
from PIL import Image, ImageTk
from tkinter import messagebox, Frame

def transform_input(Dataset):
    loader = DataLoader(Dataset, batch_size=100, num_workers=0 , drop_last= True, shuffle= True) 
    return loader

# ... [Inserisci le tue definizioni precedenti per modello, dataset, ecc. qui] ...

class BellomoApp(tk.Tk):

    def __init__(self):
        super().__init__()

        self.title("Demo")
        self.geometry("260x480")

        self.frame = Frame(self, bg="#f5f5f5")
        self.frame.place(relwidth=1, relheight=1)

        self.canvas = tk.Canvas(self.frame, bg="gray", width=224, height=224)
        self.canvas.pack(pady=20)

        self.prediction_label = tk.Label(self.frame, text="", font=('Arial', 12), bg="#f5f5f5")
        self.prediction_label.pack(pady=10)

        self.btn_run_demo = tk.Button(self.frame, text="Run Demo", command=self.call_demo)
        self.btn_run_demo.pack(pady=10)

        device = "cuda" if torch.cuda.is_available() else "cpu"
        print(f"Using device: {device}")

        try:
            self.model = BellomoClassifier().eval()
            self.model.load_state_dict(torch.load('%s-%d.pth' % ("weight/BellomoClassifier-5", 55), map_location=torch.device(device)))
            self.model.to(device)
        except Exception as e:
            print(f"Error loading the model: {e}")

    def call_demo(self):
        loader = transform_input(test)  # Assuming this function returns DataLoader
        self.demo(loader, 1)

    def demo(self, loader, q):
        i = 0
        device = "cuda" if torch.cuda.is_available() else "cpu"
        
        classes = cDict().get()

        for batch in loader:
            images = batch['image']
            labels = batch['label']

            x = images.to(device)
            y = labels.to(device)

            output = self.model(x)
            preds = output.to('cpu').max(1)[1].numpy()
            labs = y.to('cpu').numpy()

            # Display the image on canvas
            img = np.transpose(images[0].numpy(), (1, 2, 0))
            img = s * img + m 
            img = np.clip(img, 0, 1)
            self.display_image_on_canvas(img)

            # Display the original and predicted label in GUI
            prediction_text = f"Predicted: {classes[preds[0]]}\nOriginal: {classes[labs[0]]}"
            self.display_prediction_text(prediction_text)

            i += 1

            if i == q:
                break

    def display_image_on_canvas(self, img_np_array):
        img = Image.fromarray((img_np_array * 255).astype(np.uint8))
        photo = ImageTk.PhotoImage(img)
        self.canvas.image = photo
        self.canvas.create_image(0, 0, anchor=tk.NW, image=photo)

    def display_prediction_text(self, prediction_text):
        self.prediction_label.config(text=prediction_text)

if __name__ == '__main__':
    app = BellomoApp()
    app.mainloop()


Using device: cuda
